<a href="https://colab.research.google.com/github/YashNigam65/gitfolder/blob/master/assignment_1_pdf_summarization_via_mistral.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Use any other open source model in place of openai and do the summarization of a pdf document using langchain and the concept of RAG. use chromadb vector database for this.

In [1]:
!pip install langchain_community
!pip install langchain

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.5 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [2]:
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.3/328.3 kB 6.8 MB/s eta 0:00:00


In [3]:
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.7/21.7 MB 133.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 109.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 139.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.0/220.0 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 10.8 MB/s

In [4]:
!pip install langchain-mistralai
import chromadb
import os
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_mistralai import ChatMistralAI, MistralAIEmbeddings

In [9]:
# Upload your PDF file to the Colab environment here (e.g., drag and drop it into the files tab)
# Once uploaded, specify the file path
pdf_file_path = "/content/sample_data/sample_test_pdf.pdf"  # <--- REPLACE WITH YOUR PDF FILE NAME

from google.colab import userdata
mistral_key = userdata.get('MISTRAL')

# Set your Mistral AI API key (replace with your actual key)
os.environ["MISTRAL_API_KEY"] = mistral_key # <--- REPLACE WITH YOUR MISTRAL AI API KEY

# Initialize Mistral embeddings
mistral_embeddings = MistralAIEmbeddings()

# Load the PDF document
loader = PyPDFLoader(pdf_file_path)
docs = loader.load()

# Split the document into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splitted_docs = text_splitter.split_documents(docs)

# Create a Chroma vector database from the document chunks
vectorstore = Chroma.from_documents(documents=splitted_docs, embedding=mistral_embeddings)

print(f"Loaded {len(docs)} pages from PDF and split into {len(splitted_docs)} chunks.")
print("Vector database created successfully.")

Loaded 1 pages from PDF and split into 2 chunks.
Vector database created successfully.


In [19]:
# ----- Retrieval and Generation Process -----

# this is an example of a user question (query)
query_one = 'Which company name mentioned in the document?'
query_two = 'Can you tell me about TCS?'
query = "Can you summerize content of pdf in 50 words"

# retrieve context - top 5 most relevant (closests) chunks to the query vector
# (by default Langchain is using cosine distance metric)
docs_chroma_one = vectorstore.similarity_search_with_score(query_one, k=5)
docs_chroma_two = vectorstore.similarity_search_with_score(query_two, k=5)

# generate an answer based on given user query and retrieved context information
context_text_one = "\n\n".join([doc.page_content for doc, _score in docs_chroma_one])
print(context_text_one)

context_text_two = "\n\n".join([doc.page_content for doc, _score in docs_chroma_two])
print(context_text_two)


HCL Technologies is a leading global IT services and consulting company 
headquartered in India, widely recognized for its innovation-driven approach, 
engineering excellence, and customer-focused solutions. Founded in 1976 as part of 
the HCL Group, the company has grown into a global technology powerhouse with a 
workforce of over 500,000 employees spread across multiple countries. This vast 
talent base enables HCL to deliver large-scale, complex digital solutions to enterprises 
worldwide. 
HCL Technologies operates across multiple streams of work, catering to diverse 
business and technology needs. Its key service areas include cloud computing, API 
development, application development and maintenance, data engineering, 
artiﬁcial intelligence, cybersecurity, DevOps, infrastructure management, and 
enterprise digital transformation. In the cloud domain, HCL helps organizations 
migrate, modernize, and optimize workloads across major cloud platforms such as 
AWS, Microsoft Azure, a

In [15]:
# you can use a prompt template
PROMPT_TEMPLATE = """
Answer the question based only on the following context:
{context}
Answer the question based on the above context: {question}.
Provide a detailed answer.
Don’t justify your answers.
Don’t give information not mentioned in the CONTEXT INFORMATION.
Do not say "according to the context" or "mentioned in the context" or similar.
"""

# load retrieved context and user query in the prompt template
prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
prompt_one = prompt_template.format(context=context_text_one, question=query_one)


# call LLM model to generate the answer based on the given context and query
model = ChatMistralAI()
response_text_one = model.invoke(prompt_one)

print(response_text_one)

content='The company name mentioned in the document is **HCL Technologies**.' additional_kwargs={} response_metadata={'token_usage': {'prompt_tokens': 874, 'total_tokens': 888, 'completion_tokens': 14}, 'model_name': 'mistral-small', 'model': 'mistral-small', 'finish_reason': 'stop', 'model_provider': 'mistralai'} id='lc_run--019b16cc-2821-7c02-a84d-aee8984a2408-0' usage_metadata={'input_tokens': 874, 'output_tokens': 14, 'total_tokens': 888}


In [16]:
prompt_two = prompt_template.format(context=context_text_two, question=query_two)
response_text_two = model.invoke(prompt_two)
print(response_text_two)

content='The context provided does not contain any information about TCS (Tata Consultancy Services). Therefore, I cannot provide details about TCS based on the given context.' additional_kwargs={} response_metadata={'token_usage': {'prompt_tokens': 874, 'total_tokens': 908, 'completion_tokens': 34}, 'model_name': 'mistral-small', 'model': 'mistral-small', 'finish_reason': 'stop', 'model_provider': 'mistralai'} id='lc_run--019b16cc-2c9a-7c71-9d49-4d25234667f7-0' usage_metadata={'input_tokens': 874, 'output_tokens': 34, 'total_tokens': 908}


In [21]:

prompt = prompt_template.format(context=context_text_one, question=query)
response_text = model.invoke(prompt)
print(response_text)

content='HCL Technologies is a global IT services and consulting firm headquartered in India, founded in 1976. With over 500,000 employees, it offers cloud computing, AI, cybersecurity, and digital transformation services. It supports major cloud platforms like AWS, Azure, and Google Cloud, modernizes legacy systems, and follows an "Employees First, Customers Second" philosophy. It operates across North America, Europe, Asia-Pacific, and the Middle East, serving industries like banking, healthcare, and manufacturing.' additional_kwargs={} response_metadata={'token_usage': {'prompt_tokens': 878, 'total_tokens': 988, 'completion_tokens': 110}, 'model_name': 'mistral-small', 'model': 'mistral-small', 'finish_reason': 'stop', 'model_provider': 'mistralai'} id='lc_run--019b16d3-df51-7252-8950-e198544a2d35-0' usage_metadata={'input_tokens': 878, 'output_tokens': 110, 'total_tokens': 988}
